In [1]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# -------------------------
#   To-do
# -------------------------
# 1. add logger
# 2. make image drawer overlay mask on image
# 3. make iterable
# 4. make verbose turn on and off
# 5. write pydoc

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ReduceLROnPlateau

# tensorflow Module
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal

# python basic Module
import os
from datetime import datetime

import numpy as np

from gan_module.data_loader.medical_segmentation_data_loader import DataLoader

from gan_module.model.build_model import build_generator
from gan_module import custom_loss
from gan_module.custom_loss import dice_loss_for_training, f1_score
from gan_module.config import CONFIG


custom_loss.AXIS = [1, 2]
USE_GPU = True
# set GPU memory growth allocation
if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


class UnetSegmentation:
    def __init__(
        self,
        generator_power=32,
        generator_learning_rate=1e-4,
        on_memory=True,
        code_test=False
    ):

        img_shape = CONFIG["img_shape"]
        input_channels = CONFIG["input_channels"]
        output_channels = CONFIG["output_channels"]

        input_img_shape = (*img_shape, input_channels)

        # Configure data loader
        self.dataset_name = "tumor"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            on_memory=on_memory, 
            code_test=code_test,
            config_dict=CONFIG
        )

        self.loaded_data_index = {
            "train": np.arange(self.data_loader.data_length["train"]),
            "valid": np.arange(self.data_loader.data_length["valid"])
        }

        # Number of filters in the first layer of G and D
        self.generator_power = generator_power
        self.generator_learning_rate = generator_learning_rate
        generator_optimizer = Nadam(self.generator_learning_rate)

        # layer Component
        self.kernel_initializer = RandomNormal(mean=0.0, stddev=0.02)

        # Build the generator
        self.generator = build_generator(
            input_img_shape=input_img_shape,
            output_channels=output_channels,
            generator_power=self.generator_power,
            kernel_initializer=self.kernel_initializer,
        )
        # loss = sm.losses.bce_dice_loss
        self.generator.compile(
            loss=dice_loss_for_training,
            optimizer=generator_optimizer,
            metrics=[f1_score],
        )
        self.generator.trainable = False
        
    def train(self, epochs, batch_size=10, start_epoch=0):

        start_time = datetime.now()

        reduce_lr = LearningRateScheduler(self.learning_rate_scheduler)
        save_c = ModelCheckpoint(
            "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5", 
            monitor='val_loss', 
            verbose=0, 
            save_best_only=False, 
            save_weights_only=True, 
            mode='min')

        reduceLROnPlat = ReduceLROnPlateau(
              monitor="val_loss",
              factor=0.1,
              patience=20,
              verbose=0,
              mode="auto",
              min_delta=0.0001,
              cooldown=5,
              min_lr=1e-7)
        csv_logger = CSVLogger('./U_net/log.csv', append=False, separator=',')

        self.generator.fit(
            x=self.data_loader.loaded_data_object["train"]["input"],
            y=self.data_loader.loaded_data_object["train"]["output"],
            validation_data=list(self.data_loader.loaded_data_object["valid"].values()),
            batch_size=batch_size, epochs=epochs,
            callbacks=[reduceLROnPlat, save_c, csv_logger],
            initial_epoch=start_epoch
        )

        elapsed_time = datetime.now() - start_time
        print(f"elapsed_time : {elapsed_time}")

    def learning_rate_scheduler(self, epoch,
                                schedule_list=None, exponent=0.2,
                                warm_up=True, warm_up_epoch=10):
        step = 0
        if warm_up and epoch < warm_up_epoch:
            new_learning_rate = self.generator_learning_rate * \
                ((epoch + 1) / warm_up_epoch)
        else:
            if schedule_list is None:
                schedule_list = [30, 100, 175, 250, 325]
            for step, target_epoch in enumerate(schedule_list):
                if target_epoch > epoch:
                    break
                else:
                    continue
            new_learning_rate = self.generator_learning_rate * \
                (exponent**(step))

        return new_learning_rate


{'img_shape': [512, 512], 'input_channels': 3, 'output_channels': 1}


In [2]:
generator_lr = 1e-3
batch_size = 4
g_lr = generator_lr * batch_size
gan = UnetSegmentation(generator_power=4, generator_learning_rate=g_lr, code_test=False)

In [3]:
#gan.find_error = True
#gan.find_error_epoch = 5
gan.train(epochs=325, batch_size=batch_size, start_epoch=0)

Epoch 1/325
1550/1550 [==============================] - 400s 258ms/step - loss: 0.7161 - f1_score: 0.4150 - val_loss: 0.0017 - val_f1_score: 0.0000e+00
Epoch 2/325
1550/1550 [==============================] - 400s 258ms/step - loss: 0.5923 - f1_score: 0.5181 - val_loss: 0.0049 - val_f1_score: 0.0000e+00
Epoch 3/325
1550/1550 [==============================] - 402s 259ms/step - loss: 0.4959 - f1_score: 0.5739 - val_loss: 0.0047 - val_f1_score: 0.0000e+00
Epoch 4/325
1550/1550 [==============================] - 402s 259ms/step - loss: 0.4453 - f1_score: 0.5938 - val_loss: 0.0053 - val_f1_score: 0.0000e+00
Epoch 5/325
1550/1550 [==============================] - 400s 258ms/step - loss: 0.4254 - f1_score: 0.5997 - val_loss: 0.0044 - val_f1_score: 0.0000e+00
Epoch 6/325
1550/1550 [==============================] - 405s 261ms/step - loss: 0.4104 - f1_score: 0.6078 - val_loss: 0.0053 - val_f1_score: 0.0000e+00
Epoch 7/325
1550/1550 [==============================] - 410s 265ms/step - loss: 0

KeyboardInterrupt: 

In [15]:
temp = list(gan.data_loader.loaded_data_object["valid"].values())

In [18]:
temp[1].shape

(600, 512, 512, 1)

In [17]:
temp[0].shape

(600, 512, 512, 3)

In [1]:
from gan_module.model.build_model import build_discriminator

temp = build_discriminator(
            input_img_shape=(512,512,3),
            output_img_shape=(512,512,1),
            discriminator_power=4,
)

0
1
2
3
4
5


In [2]:
temp.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 512, 512, 4)  0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 4)  148         concatenate[0][0]     